# Drive link for dataset and csv files
https://drive.google.com/drive/folders/1Y9Dr5OQJl5i1WuMNwobfBOc5AEfugEcC?usp=sharing


In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install easyocr -q
import pandas as pd
import easyocr
import re
import csv
import cv2
import os
import numpy as np
from google.colab import files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 20.9 MB/s eta 0:00:00


In [2]:
reader = easyocr.Reader(['en'], gpu=True)
parameter_patterns = {
    "weight": r"(\d+\s*gm|\d+\s*g|\d+\s*kg)",  # Match numbers followed by 'gm' or 'g'
    "height": r"(\d+\s*(cm|mm|inch|in|feet|ft))",  # Match numbers followed by height units
    "depth": r"(\d+\s*(cm|mm|inch|in|feet|ft))",   # Same as height (cm, mm, inches, feet)
    "width": r"(\d+\s*(cm|mm|inch|in|feet|ft))",   # Same as height (cm, mm, inches, feet)
    "item_volume": r"(\d+\s*(cup|gallon|millilitre|ml|oz|fl\.oz|fluid ounce))",  # Volume units
    "item_weight": r"(\d+\s*(grams|g|kilograms|kg|milligrams|mg|pound|lb|ton|oz|ounce))",  # Weight units
    "voltage": r"(\d+\s*volts?|\d+\s*V)",  # Match numbers followed by 'volt' or 'volts'
    "wattage": r"(\d+\s*watts?|\d+\s*W)",  # Match numbers followed by 'watt' or 'watts'
    "price": r"(\$\d+|\d+\s*\$)",   # Match price formats (e.g., $100)
    "maximum_weight_recommendation": r"(\d+\s*(grams|g|kilograms|kg|milligrams|mg|pound|lb|ton|oz|ounce))"
}

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

In [3]:
# Path to your CSV file and image dataset folder in Google Colab
csv_file = '/content/test_final.csv'
image_dataset_folder = '/content/drive/MyDrive/amazon_db/amazon_final_db/part 2'  # Update this to the correct path where your images are stored

# Dictionary to store entity values for each image
entity_name_dict = {}

# Read the CSV and create a dictionary mapping image names to entity values
def load_entity_values_from_csv(csv_file):
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)

        # Loop through each row in the CSV
        for row in reader:
            image_name = row['image_name']  # First column: image file name
            entity_name = row['entity_name']  # Third column: entity value to predict
            # Store the entity value in a dictionary with the image name as the key
            entity_name_dict[image_name] = entity_name

# Function to find and print the entity value for each image in the dataset
def find_entity_values_for_images(image_dataset_folder, entity_name_dict):
    # List all the images in the dataset folder
    image_files = os.listdir(image_dataset_folder)
    # Loop through each image in the dataset
    for image_file in image_files:
        # Check if the image file is listed in the CSV
        if image_file in entity_name_dict:
            entity_name = entity_name_dict[image_file]
            print(f"Image: {image_file}, Entity Name: {entity_name}")
        else:
            print(f"Entity name not found for image: {image_file}")

# Load entity values from the CSV file
load_entity_values_from_csv(csv_file)

# Find and print entity values for each image in the dataset
find_entity_values_for_images(image_dataset_folder, entity_name_dict)

def classify_line_orientation(line):
            x1, y1, x2, y2 = line[0]
            angle = np.arctan2(y2 - y1, x2 - x1) * 180.0 / np.pi
            if -45 <= angle <= 45:  # Horizontal lines
                return 'width'
            elif 45 < angle <= 135 or -135 <= angle < -45:  # Vertical lines
                return 'height'
            else:
                return 'depth'
def contains_numbers(text):
    return bool(re.search(r'\d', text))

Streaming output truncated to the last 5000 lines.
Image: 51607-hxmOL.jpg, Entity Name: item_weight
Image: 5160QN+FIeL.jpg, Entity Name: item_volume
Image: 5160RTtsi3L.jpg, Entity Name: height
Image: 5160X0g02nL.jpg, Entity Name: voltage
Image: 5160XVS9S-L.jpg, Entity Name: depth
Image: 5160ZGWrd2L.jpg, Entity Name: width
Image: 5160ciUml2L.jpg, Entity Name: height
Image: 5160fp2Vx6L.jpg, Entity Name: wattage
Image: 5160gbQyezL.jpg, Entity Name: height
Image: 5160liGmuSL.jpg, Entity Name: voltage
Image: 5160qfsqWdL.jpg, Entity Name: width
Image: 5160sbXmlxL.jpg, Entity Name: item_weight
Image: 516154ueQPL.jpg, Entity Name: item_weight
Image: 51615qLLn+L.jpg, Entity Name: depth
Image: 5161AKECQcL.jpg, Entity Name: height
Image: 5161Z9bBUmL.jpg, Entity Name: width
Image: 5161aw1SXWL.jpg, Entity Name: item_weight
Image: 5161kcGG6IL.jpg, Entity Name: item_weight
Image: 5161oI2MRML.jpg, Entity Name: voltage
Image: 5161qEVuQJL.jpg, Entity Name: maximum_weight_recommendation
Image: 5161rlxpcL

In [6]:
output_data = []
def extract_parameter_from_image(filename, entity_name):
  if entity_name.lower() not in parameter_patterns:
      print(f"No pattern defined for the parameter: {entity_name}")
      return
  file_path = os.path.join(image_dataset_folder, filename)
  if not os.path.exists(file_path):
    print(f"File not found: {file_path}")  # Print an error message if the file doesn't exist
    return
  # Read the image and perform OCR
  text = reader.readtext(file_path)

  # Get the pattern for the specified parameter
  pattern = parameter_patterns[entity_name.lower()]

  # Iterate through detected text and filter based on the parameter pattern
  extracted_values = []
  for i in text:
    detected_text = i[1]  # Extract the detected text from the tuple
    matches = re.findall(pattern, detected_text)
    if matches:
      extracted_value = matches[0][0]

      # Insert space between number and unit if they are joined (e.g., '53cm' -> '53 cm')
      extracted_value = re.sub(r'(\d+)([a-zA-Z]+)', r'\1 \2', extracted_value)
      extracted_values.append(extracted_value)  # Append the result

  # Output the result and store it
  if extracted_values:
    output_data.append(extracted_values[0])  # Store the first value
    print(f"File {file_path} has {entity_name}: {extracted_values[0]}")
  else:
    output_data.append("2 grams")  # Default value when no match is found
    print("2 grams")
    #print(f"File {file_path} has no info about {entity_name}")

In [ ]:
for filename, entity_name in entity_name_dict.items():
  if entity_name in ['length', 'width', 'depth']:
    file_path = os.path.join(image_dataset_folder, filename)
    img = cv2.imread(file_path)
    if img is None:
      print(f"Image not found or could not be loaded: {file_path}")
      continue # Skip to the next image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)
    if lines is None:
      print(f"No lines detected for file: {filename}")
      continue
    ocr_results = reader.readtext(gray)
    lablez=[]
    dims=[]
    for line in lines:
        x1, y1, x2, y2 = line[0]
        dimension_type = classify_line_orientation(line)
        line_midpoint = ((x1 + x2) // 2, (y1 + y2) // 2)
        for (bbox, text, prob) in ocr_results:
            (top_left, top_right, bottom_right, bottom_left) = bbox
            text_midpoint = ((top_left[0] + bottom_right[0]) // 2, (top_left[1] + bottom_right[1]) // 2)
            distance = np.sqrt((line_midpoint[0] - text_midpoint[0]) ** 2 + (line_midpoint[1] - text_midpoint[1]) ** 2)
            if distance < 200:  # Threshold for associating text with a line
              if text not in lablez and contains_numbers(text):  # Only consider text with numbers
                  lablez.append(text)
                  text = text.split('/')
                  if dimension_type not in dims and entity_name == dimension_type:
                      dims.append(dimension_type)
                      entity_value = re.sub(r'(\d+)([a-zA-Z]+)', r'\1 \2', text[0]) #add space
                      output_data.append(entity_value)
                      print(f"File name: {filename} Detected text: {text[0]} classified as {dimension_type}")
                  else:
                      continue
  #print("\n")
  else:
    extract_parameter_from_image(filename, entity_name)
df = pd.DataFrame({
    'index': range(len(output_data)),
    'predictions': output_data
})

# Define the path to save the CSV file in Google Colab
csv_file_path = '/content/predictions.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

print(f"CSV file saved at: {csv_file_path}")
files.download('/content/predictions.csv')


Streaming output truncated to the last 5000 lines.
File name: 516D8Gsx1RL.jpg Detected text: 13cm classified as width
File name: 516DBYZB9wL.jpg Detected text: G8-Bi-Pin Base classified as width
File name: 516DIWg+luL.jpg Detected text: 25mm classified as width
File /content/drive/MyDrive/amazon_db/amazon_final_db/part 2/516DU9bqSAL.jpg has height: 7 cm
2 grams
File name: 516DnjulWoL.jpg Detected text: 6OCM classified as width
File /content/drive/MyDrive/amazon_db/amazon_final_db/part 2/516DqelVEiL.jpg has height: 47 cm
File /content/drive/MyDrive/amazon_db/amazon_final_db/part 2/516E5+hWYfS.jpg has height: 5 cm
2 grams
2 grams
File name: 516EbrYJmJL.jpg Detected text: 22.5cm classified as width
2 grams
File name: 516EpX5rmJS.jpg Detected text: 1000 classified as width
File /content/drive/MyDrive/amazon_db/amazon_final_db/part 2/516EwRmkUnL.jpg has height: 153 cm
File /content/drive/MyDrive/amazon_db/amazon_final_db/part 2/516EyFNOKnL.jpg has height: 76 cm
File name: 516F5Jpg0tS.jpg De

In [ ]:
  extract_parameter_from_image(filename, entity_name)
df = pd.DataFrame({
    'index': range(len(output_data)),
    'predictions': output_data
})

# Define the path to save the CSV file in Google Colab
csv_file_path = '/content/predictions.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

print(f"CSV file saved at: {csv_file_path}")
files.download('/content/predictions.csv')